In [1]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)

print("Torch version:", torch.__version__)


Torch version: 2.1.2+cu121


In [2]:
# create own Dataset
my_dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)

# own DataLoader
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)


model = get_model_instance_segmentation(config.num_classes)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(data_loader)

[{'boxes': tensor([[214.0000, 379.0000, 254.5000, 424.5000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([21], device='cuda:0'), 'area': tensor([1842.7500], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[209.0000, 272.0000, 266.5000, 378.5000],
        [341.0000, 273.0000, 398.5000, 385.0000]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0'), 'image_id': tensor([18], device='cuda:0'), 'area': tensor([6123.7500, 6440.0000], device='cuda:0'), 'iscrowd': tensor([0, 0], device='cuda:0')}]
[{'boxes': tensor([[125.0000, 407.0000, 287.0000, 585.5000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([8], device='cuda:0'), 'area': tensor([28917.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[211.0000, 259.0000, 275.5000, 398.0000],
        [342.0000, 258.0000, 409.5000, 392.5000]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0'), 'image_id':

In [4]:
# Training
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        # Debug
        # print(annotations)
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

Epoch: 0/10
[{'boxes': tensor([[124.0000,  54.0000, 278.5000, 232.0000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([35], device='cuda:0'), 'area': tensor([27501.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
Iteration: 1/48, Loss: 1.354773998260498
[{'boxes': tensor([[217.0000, 214.0000, 255.0000, 266.5000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([48], device='cuda:0'), 'area': tensor([1995.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
Iteration: 2/48, Loss: 1.2143312692642212
[{'boxes': tensor([[159., 433., 291., 575.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([42], device='cuda:0'), 'area': tensor([18744.], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
Iteration: 3/48, Loss: 1.0243011713027954
[{'boxes': tensor([[170.0000,  57.0000, 288.5000, 199.5000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tens